<a href="https://colab.research.google.com/github/YashDarapu/Sentiment-Analysis-Major-Project/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 9.1MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [4]:
df=pd.read_csv('/content/Train.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


177

In [6]:
# removing html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [7]:
# expand contractions
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

     |████████████████████████████████| 327kB 6.1MB/s 
     |████████████████████████████████| 266kB 14.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=85266 sha256=dd096c408b9edb47bc4cd7c8ce852bac419f7a6c164be32c58757f62c71e0939
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [8]:
# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [9]:
# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [10]:
# 1: lower case
# 2: html tag
# 3: contraction
# 4: tokenize
# 5: stop words
df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

df.head()

,id,label,tweet
0,1,0,user father dysfunctional selfish drags kids d...
1,2,0,user user thanks lyft credit not use not offer...
2,3,0,bihday majesty
3,4,0,model love take time ur
4,5,0,factsguide society motivation


In [11]:
# df['compound']=df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])
df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [12]:
conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

# display updated DataFrame
df

,id,label,tweet,compound,tier
0,1,0,user father dysfunctional selfish drags kids d...,-0.7650,Negative
1,2,0,user user thanks lyft credit not use not offer...,0.6705,Positive
2,3,0,bihday majesty,0.0000,Neutral
3,4,0,model love take time ur,0.6369,Positive
4,5,0,factsguide society motivation,0.3400,Positive
...,...,...,...,...,...
9995,9996,0,user routine whack evening walk walk beach sun...,0.5423,Positive
9996,9997,0,I dead still happy poledance madrid fitness ma...,0.1280,Positive
9997,9998,0,united kingdom claimant count rate 22 may prev...,0.4215,Positive
9998,9999,0,rip friend shocked dismay heabroken cannotbeli...,-0.7096,Negative


In [13]:
df.head()

,id,label,tweet,compound,tier
0,1,0,user father dysfunctional selfish drags kids d...,-0.7650,Negative
1,2,0,user user thanks lyft credit not use not offer...,0.6705,Positive
2,3,0,bihday majesty,0.0000,Neutral
3,4,0,model love take time ur,0.6369,Positive
4,5,0,factsguide society motivation,0.3400,Positive


In [14]:
x = df.iloc[:,2].values
y = df.iloc[:,-1].values

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0,stratify = y)
x_train.shape

(7000,)

In [16]:
np.unique(y_train,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([1460, 1903, 3637]))

In [17]:
np.unique(y_test,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([ 626,  816, 1558]))

In [18]:
# Pipeline = Combine above 2 steps
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
# Syntax : Pipeline([('Variable 1',Method 1()),('Variable 2',Method 2())])

In [19]:
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

In [20]:
text_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [21]:
y_pred = text_model.predict(x_test)
y_pred

array(['Positive', 'Positive', 'Neutral', ..., 'Negative', 'Positive',
       'Positive'], dtype=object)

In [22]:
np.unique(y_pred,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([ 402,  799, 1799]))

In [23]:
y_test

array(['Positive', 'Neutral', 'Neutral', ..., 'Negative', 'Positive',
       'Neutral'], dtype=object)

In [25]:
# Evaluation
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [26]:
accuracy_score(y_pred,y_test)*100

75.73333333333333

In [27]:
confusion_matrix(y_pred,y_test)

array([[ 313,   30,   59],
       [ 121,  569,  109],
       [ 192,  217, 1390]])

In [28]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    Negative       0.50      0.78      0.61       402
     Neutral       0.70      0.71      0.70       799
    Positive       0.89      0.77      0.83      1799

    accuracy                           0.76      3000
   macro avg       0.70      0.75      0.71      3000
weighted avg       0.79      0.76      0.77      3000



In [29]:
text_model.predict(["#people aren't protesting #trump because a #republican not every on e won never hw is dumbo kjkjwe @#$%^&*Â€Â¦ "])

array(['Positive'], dtype=object)

In [30]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 7.5MB 9.3MB/s 
     |████████████████████████████████| 163kB 45.1MB/s 
     |████████████████████████████████| 4.6MB 43.2MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 112kB 46.9MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 122kB 39.4MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=6ebaa6c396be02f26e3469ce5757175f1c9af02dd3c38f9102bf5a648ae935b7
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=72f3613b429b6d530022d058822805b62b597ed41c71444a57c2d373d5603007
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [33]:
%%writefile app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

# remove html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

# !pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text


# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text


st.title("Tweet's Sentiment Analysis")

df = pd.read_csv('/content/Train.csv')

df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

x = df.iloc[:,2].values
y = df.iloc[:,-1].values

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 
text_model.fit(x,y)
select = st.text_input('Enter your message')

if(st.markdown(
    '<span class="badge badge-pill badge-success"> Badge </span>',
    unsafe_allow_html=True
)):
  op = text_model.predict([select])
  ans=op[0]

  if ans == 'Positive':
    st.success("Positive 🙂")
  if ans == 'Negative':
    st.error("Negative 😠")
  if ans== 'Neutral':
    st.warning("Neutral 😐")

Overwriting app.py


In [32]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://f0405f884f7b.ngrok.io'